In [1]:
!pip install jproperties

# Importing modules and loading data



In [2]:
import psycopg2 as sql
import pickle
import glob
from tqdm import tqdm 
import numpy as np
import pandas as pd
import re
import json
import flask

from jproperties import Properties

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
with open('/content/drive/MyDrive/config/config_RDS.prop','rb') as f:
  p = Properties()

  p.load(f,'utf-8')

In [ ]:
# Load DF into RAM
with open('/content/drive/MyDrive/Pickle files/Product_level_dataframe','rb') as f:
  df = pickle.load(f)

In [ ]:
# Load DF into RAM
with open('/content/drive/MyDrive/Pickle files/final_data_frame','rb') as f:
  image_df = pickle.load(f)

In [ ]:
df

,gender,category,sub-category,product-number,product-name,path
0,women,dresses,casual_and_day_dresses,56037632,black knit midi dress,women/dresses/casual_and_day_dresses/56037632/...
8,women,dresses,casual_and_day_dresses,62629380,multicolor henley t-shirt dress,women/dresses/casual_and_day_dresses/62629380/...
11,women,dresses,casual_and_day_dresses,67938089,black stitch dress,women/dresses/casual_and_day_dresses/67938089/...
20,women,dresses,casual_and_day_dresses,67981504,blue signature slim short-sleeve shirtdress,women/dresses/casual_and_day_dresses/67981504/...
32,women,dresses,casual_and_day_dresses,70969922,multicolor draped floral-print dress,women/dresses/casual_and_day_dresses/70969922/...
...,...,...,...,...,...,...
201826,women,tops,t-shirts,91437497,black fight party t-shirt,women/tops/t-shirts/91437497/91437497_1.jpeg
201830,women,tops,t-shirts,91437517,classic tee black,women/tops/t-shirts/91437517/91437517_1.jpeg
201832,women,tops,t-shirts,91437722,black red heart t-shirt,women/tops/t-shirts/91437722/91437722_0.jpeg
201833,women,tops,t-shirts,91437753,white run shirt,women/tops/t-shirts/91437753/91437753_0.jpeg


In [ ]:
max([len(x) for x in df['product-name']])

95

In [ ]:
image_df

,gender,category,sub-category,product-number,product-name,path
0,women,dresses,casual_and_day_dresses,56037632,black knit midi dress,women/dresses/casual_and_day_dresses/56037632/...
3,women,dresses,casual_and_day_dresses,56037632,black knit midi dress,women/dresses/casual_and_day_dresses/56037632/...
4,women,dresses,casual_and_day_dresses,56037632,black knit midi dress,women/dresses/casual_and_day_dresses/56037632/...
6,women,dresses,casual_and_day_dresses,56037632,black knit midi dress,women/dresses/casual_and_day_dresses/56037632/...
8,women,dresses,casual_and_day_dresses,62629380,multicolor henley t-shirt dress,women/dresses/casual_and_day_dresses/62629380/...
...,...,...,...,...,...,...
201831,women,tops,t-shirts,91437517,classic tee black,women/tops/t-shirts/91437517/91437517_2.jpeg
201832,women,tops,t-shirts,91437722,black red heart t-shirt,women/tops/t-shirts/91437722/91437722_0.jpeg
201833,women,tops,t-shirts,91437753,white run shirt,women/tops/t-shirts/91437753/91437753_0.jpeg
201834,women,tops,t-shirts,91437753,white run shirt,women/tops/t-shirts/91437753/91437753_1.jpeg


In [ ]:
df[df['product-number']=='71369781']

,gender,category,sub-category,product-number,product-name,path


# Creating SQL connection and setting up insert DB

In [9]:
conn = sql.connect(
    host = p.get('host').data,
    user = p.get('user').data,
    password = p.get('password').data,
    port = p.get('port').data,
    dbname = p.get('dbname').data)

In [ ]:
def insert_row(stuff):
  
  cursor.executemany("update products set product_name = %s where product_number = %s", stuff)
  conn.commit()

In [ ]:
stuff = list(zip(df['product-name'],df['product-number']))
stuff[:10]

[('black knit midi dress', '56037632'),
 ('multicolor henley t-shirt dress', '62629380'),
 ('black stitch dress', '67938089'),
 ('blue signature slim short-sleeve shirtdress', '67981504'),
 ('multicolor draped floral-print dress', '70969922'),
 ('gray wrap dress', '71070512'),
 ("blue women's embroidered midi-dress", '71350683'),
 ('red print shirt dress', '71369781'),
 ('teal cotton gingham shirt dress', '71390126'),
 ('gray tie collar dress', '71572105')]

In [ ]:
cursor = conn.cursor()
insert_row(stuff)
cursor.close()
conn.close()

In [ ]:
values = []
i = 1
j = 1
p_no = []
cursor = conn.cursor()
for row in tqdm(df.itertuples()):
  values.append(row._5)
  p_no.append(row._4)
  i+=1
  if i%1001 == 0 and i != 0 :
    #print('in_if')
    insert_row(flask.escape(values),p_no)
    #print(len(values))
    values = []
    P_no = []
    i = 1
    #print(j)
    j+=1
  #print(dir(row))
insert_row(values,p_no)
#print(len(values))
#print('inside else')

cursor.close()
conn.close()

0it [00:00, ?it/s]


TypeError: ignored

In [ ]:
index = pd.DataFrame(np.arange(1,df.shape[0]+1))
df.reset_index(drop=True,inplace=True)
df = pd.concat([df,index],axis=1)
df.rename(columns={0:'product_id'},inplace=True)


In [ ]:
image_df['is_main_image'] = np.where(image_df['path'].isin(df['path']), True, False)
image_df

,gender,category,sub-category,product-number,product-name,path,is_main_image
0,women,dresses,casual_and_day_dresses,56037632,black knit midi dress,women/dresses/casual_and_day_dresses/56037632/...,True
3,women,dresses,casual_and_day_dresses,56037632,black knit midi dress,women/dresses/casual_and_day_dresses/56037632/...,False
4,women,dresses,casual_and_day_dresses,56037632,black knit midi dress,women/dresses/casual_and_day_dresses/56037632/...,False
6,women,dresses,casual_and_day_dresses,56037632,black knit midi dress,women/dresses/casual_and_day_dresses/56037632/...,False
8,women,dresses,casual_and_day_dresses,62629380,multicolor henley t-shirt dress,women/dresses/casual_and_day_dresses/62629380/...,True
...,...,...,...,...,...,...,...
201831,women,tops,t-shirts,91437517,classic tee black,women/tops/t-shirts/91437517/91437517_2.jpeg,False
201832,women,tops,t-shirts,91437722,black red heart t-shirt,women/tops/t-shirts/91437722/91437722_0.jpeg,True
201833,women,tops,t-shirts,91437753,white run shirt,women/tops/t-shirts/91437753/91437753_0.jpeg,True
201834,women,tops,t-shirts,91437753,white run shirt,women/tops/t-shirts/91437753/91437753_1.jpeg,False


In [ ]:
prod_id = dict(zip(df['product-number'],df['product_id']))
prod_id.keys()

dict_keys(['56037632', '62629380', '67938089', '67981504', '70969922', '71070512', '71350683', '71369781', '71390126', '71572105', '74114141', '74212876', '74223539', '76182637', '76318958', '76370348', '77875713', '79589744', '79858644', '87669967', '87734650', '87865462', '87892450', '87908669', '88000561', '88156739', '88220447', '88294075', '88319508', '88345844', '88383169', '88403227', '88578554', '88640865', '88643980', '88680001', '88777139', '88904523', '88926373', '88972023', '89007471', '89051175', '89099853', '89148352', '89186946', '89241136', '89330451', '89353136', '89353196', '89407913', '89443961', '89445199', '89450013', '89455797', '89484081', '89523980', '89545420', '89545524', '89548931', '89610276', '89658856', '89665516', '89743884', '89754126', '89862510', '89890683', '89968626', '90033558', '90082864', '90123721', '90142857', '90155915', '90330345', '90351739', '90362696', '90373794', '90573058', '90612003', '90618979', '90627810', '90712305', '90742543', '9084

In [ ]:
array = []
for x in image_df['product-number']:
  array.append(prod_id[x]) 

In [ ]:
array = pd.DataFrame(array)

In [ ]:
image_df.reset_index(inplace=True)
image_df = pd.concat([image_df,array],axis=1)
image_df.rename(columns={0:'product-id'},inplace=True)
image_df

,level_0,index,gender,category,sub-category,product-number,product-name,path,is_main_image,product-id
0,0,0,women,dresses,casual_and_day_dresses,56037632,black knit midi dress,women/dresses/casual_and_day_dresses/56037632/...,True,1
1,1,3,women,dresses,casual_and_day_dresses,56037632,black knit midi dress,women/dresses/casual_and_day_dresses/56037632/...,False,1
2,2,4,women,dresses,casual_and_day_dresses,56037632,black knit midi dress,women/dresses/casual_and_day_dresses/56037632/...,False,1
3,3,6,women,dresses,casual_and_day_dresses,56037632,black knit midi dress,women/dresses/casual_and_day_dresses/56037632/...,False,1
4,4,8,women,dresses,casual_and_day_dresses,62629380,multicolor henley t-shirt dress,women/dresses/casual_and_day_dresses/62629380/...,True,2
...,...,...,...,...,...,...,...,...,...,...
91466,91466,201831,women,tops,t-shirts,91437517,classic tee black,women/tops/t-shirts/91437517/91437517_2.jpeg,False,57259
91467,91467,201832,women,tops,t-shirts,91437722,black red heart t-shirt,women/tops/t-shirts/91437722/91437722_0.jpeg,True,57260
91468,91468,201833,women,tops,t-shirts,91437753,white run shirt,women/tops/t-shirts/91437753/91437753_0.jpeg,True,57261
91469,91469,201834,women,tops,t-shirts,91437753,white run shirt,women/tops/t-shirts/91437753/91437753_1.jpeg,False,57261
